In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
all_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")
area_code_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_AreaCodes.csv')

code_to_name_dict = area_code_df[['Area Code', 'Area']].set_index('Area Code').to_dict()['Area']
all_df.loc[:, 'Area'] = all_df['Area Code'].replace(code_to_name_dict)

# check country name

In [24]:
with open('./data/ne_110m.json', 'r') as f:
    text = f.read()

In [25]:
for country_name in all_df['Area'].unique():
    if text.find('"NAME":"{}"'.format(country_name)) == -1:
        print(country_name)


Antigua and Barbuda
Bahrain
Barbados
Belgium-Luxembourg
Cabo Verde
China; (including Taiwan)
China; Hong Kong SAR
China; Macao SAR
Comoros
Cook Islands
Czechoslovakia
Dominica
Eswatini
Ethiopia PDR
Faroe Islands
French Guyana
French Polynesia
Grenada
Guadeloupe
Kiribati
Maldives
Malta
Marshall Islands
Martinique
Mauritius
Micronesia (Federated States of)
Nauru
Niue
R�union
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
Sao Tome and Principe
Serbia and Montenegro
Seychelles
Singapore
South Sudan
Sudan (former)
Tokelau
Tonga
Tuvalu
USSR
Yugoslav SFR
World
Africa
Eastern Africa
Middle Africa
Northern Africa
Southern Africa
Western Africa
Americas
Northern America
Central America
Caribbean
South America
Asia
Central Asia
Eastern Asia
Southern Asia
South-eastern Asia
Western Asia
Europe
Eastern Europe
Northern Europe
Southern Europe
Western Europe
Oceania
Australia and New Zealand
Melanesia
Micronesia
Polynesia
European Union (27)
Least Developed Countries
Land Loc

# wrangle file

In [27]:
all_df[all_df['Area Code'] >= 300]['Area'].unique()

array(['China; (including Taiwan)', 'World', 'Africa', 'Eastern Africa',
       'Middle Africa', 'Northern Africa', 'Southern Africa',
       'Western Africa', 'Americas', 'Northern America',
       'Central America', 'Caribbean', 'South America', 'Asia',
       'Central Asia', 'Eastern Asia', 'Southern Asia',
       'South-eastern Asia', 'Western Asia', 'Europe', 'Eastern Europe',
       'Northern Europe', 'Southern Europe', 'Western Europe', 'Oceania',
       'Australia and New Zealand', 'Melanesia', 'Micronesia',
       'Polynesia', 'European Union (27)', 'Least Developed Countries',
       'Land Locked Developing Countries',
       'Small Island Developing States',
       'Low Income Food Deficit Countries',
       'Net Food Importing Developing Countries'], dtype=object)

In [37]:
all_df['Element'].unique()

array(['Area harvested', 'Yield', 'Production', 'Stocks',
       'Producing Animals/Slaughtered', 'Laying', 'Yield/Carcass Weight',
       'Milk Animals', 'Prod Popultn'], dtype=object)

In [41]:
all_df[all_df['Element'] == 'Producing Animals/Slaughtered']

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
24,2,'004,Afghanistan,869,'21512,"Cattle fat, unrendered",5320,Producing Animals/Slaughtered,Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,480000.0,480000.0,540000.0,630000.0,722000.0,794000.0,866000.0,950000.0,828000.0,700000.0,600000.0,833000.0,761000.0,829000.0,784000.0,651700.0,761100.0,739000.0,750000.0,727800.0,766700.0,772000.0,744500.0,669717.0,619065.0,671906.0,638865.0,643279.0,651350.0,644498.0
32,2,'004,Afghanistan,868,'21151,"Edible offal of cattle, fresh, chilled or frozen",5320,Producing Animals/Slaughtered,Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,480000.0,480000.0,540000.0,630000.0,722000.0,794000.0,866000.0,950000.0,828000.0,700000.0,600000.0,833000.0,761000.0,829000.0,784000.0,651700.0,761100.0,739000.0,750000.0,727800.0,766700.0,772000.0,744500.0,669717.0,619065.0,671906.0,638865.0,643279.0,651350.0,644498.0
34,2,'004,Afghanistan,1018,'21156,"Edible offal of goat, fresh, chilled or frozen",5320,Producing Animals/Slaughtered,Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700000.0,1860000.0,2000000.0,2200000.0,2400000.0,2400000.0,2800000.0,3300000.0,3700000.0,3700000.0,2500000.0,3600000.0,3700000.0,3800000.0,3500000.0,3400000.0,2700000.0,3200000.0,2900000.0,3400000.0,3500000.0,3400000.0,3400000.0,3438257.0,3675215.0,3418681.0,3463332.0,3511962.0,3593831.0,3578963.0
36,2,'004,Afghanistan,978,'21155,"Edible offal of sheep, fresh, chilled or frozen",5320,Producing Animals/Slaughtered,Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200000.0,6800000.0,6600000.0,6600000.0,6400000.0,7100000.0,7700000.0,8200000.0,9000000.0,7600000.0,7100000.0,4500000.0,4600000.0,5100000.0,5500000.0,4700000.0,4110000.0,5430000.0,6230000.0,7100000.0,7220000.0,6940000.0,6625000.0,6844995.0,6721936.0,6740686.0,7046244.0,6904031.0,5235662.0,6918980.0
38,2,'004,Afghanistan,1128,'21159.02,"Edible offals of camels and other camelids, fr...",5320,Producing Animals/Slaughtered,Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18000.0,18000.0,18000.0,18000.0,18000.0,19000.0,20000.0,21000.0,22000.0,22000.0,20000.0,20000.0,20000.0,22000.0,21000.0,20000.0,21000.0,21000.0,22000.0,22000.0,19800.0,20000.0,19500.0,19810.0,19986.0,19955.0,20271.0,20400.0,20242.0,20323.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77343,5817,'902,Net Food Importing Developing Countries,995,'02953,Raw hides and skins of sheep or lambs,5320,Producing Animals/Slaughtered,Head,40006475.0,40120625.0,40887002.0,41927304.0,43186636.0,44653997.0,45718198.0,47182957.0,47465624.0,48191145.0,48854769.0,48252562.0,49844750.0,50934541.0,53541748.0,55568905.0,56991395.0,59303653.0,59543058.0,62981476.0,66114420.0,63342715.0,63484452.0,63179302.0,62891089.0,61825973.0,62827748.0,64513052.0,67618266.0,69385536.0,69262883.0,68885424.0,65383468.0,66634974.0,69247810.0,68470361.0,71424688.0,76947554.0,81531817.0,81565175.0,83021266.0,82293457.0,82909558.0,86038159.0,89003188.0,94667701.0,97143065.0,100948686.0,103688277.0,10722231

In [36]:
all_df[all_df['Element'] == 'Prod Popultn']['Item'].unique()

array(['Area harvested', 'Yield', 'Production', 'Stocks',
       'Producing Animals/Slaughtered', 'Laying', 'Yield/Carcass Weight',
       'Milk Animals', 'Prod Popultn'], dtype=object)

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
198,2,'004,Afghanistan,1717,'F1717,"Cereals, Total",5312,Area harvested,ha,3313500.0,3425500.0,3426500.0,3446500.0,3444500.0,3446500.0,3124500.0,3041500.0,3081500.0,3178000.0,3400000.0,3923100.0,3337000.0,3342000.0,3404000.0,3394000.0,3388000.0,3392000.0,3183500.0,3018500.0,2909000.0,2799500.0,2649000.0,2562000.0,2434500.0,2347000.0,2496000.0,2331000.0,2290000.0,2253000.0,2345071.0,2250000.0,2631882.0,2757000.0,2669450.0,2691670.0,2730401.0,2792105.0,2532871.0,2406000.0,2094000.0,2238000.0,3004700.0,2667000.0,3030000.0,2989000.0,3021000.0,2714000.0,3192000.0,2962000.0,2820000.0,3143000.0,3182922.0,3344733.0,2724070.0,2793694.0,2419097.0,1911658.0,2641911.0,3043589.0
199,2,'004,Afghanistan,1717,'F1717,"Cereals, Total",5419,Yield,hg/ha,11151.0,10790.0,9858.0,10828.0,10989.0,10123.0,12245.0,12875.0,13104.0,11051.0,9762.0,10069.0,12796.0,13019.0,13164.0,13624.0,12240.0,12919.0,13237.0,13490.0,13383.0,13302.0,13556.0,13341.0,13317.0,13140.0,13598.0,12857.0,12376.0,12006.0,11604.0,10978.0,11329.0,11404.0,12145.0,12044.0,13488.0,13880.0,12857.0,8063.0,10067.0,16698.0,14580.0,13348.0,17904.0,15517.0,19153.0,14554.0,20407.0,20111.0,16599.0,20296.0,20485.0,20175.0,21322.0,19804.0,20232.0,21626.0,21134.0,19799.0
200,2,'004,Afghanistan,1717,'F1717,"Cereals, Total",5510,Production,tonnes,3695000.0,3696000.0,3378000.0,3732000.0,3785000.0,3489000.0,3826000.0,3916000.0,4038000.0,3512000.0,3319000.0,3950000.0,4270000.0,4351000.0,4481000.0,4624000.0,4147000.0,4382000.0,4214000.0,4072000.0,3893000.0,3724000.0,3591000.0,3418000.0,3242000.0,3084000.0,3394000.0,2997000.0,2834000.0,2705000.0,2721302.0,2470000.0,2981774.0,3144000.0,3242179.0,3241915.0,3682723.0,3875506.0,3256396.0,1940000.0,2108000.0,3737000.0,4381000.0,3560000.0,5425000.0,4638000.0,5786000.0,3950000.0,6514000.0,5957000.0,4681020.0,6379000.0,6520329.0,6748023.0,5808288.0,5532695.0,4894365.0,4134191.0,5583461.0,6025977.0
532,3,'008,Albania,1717,'F1717,"Cereals, Total",5312,Area harvested,ha,347764.0,332882.0,298725.0,342158.0,323238.0,328443.0,332935.0,323150.0,322299.0,327282.0,312203.0,331220.0,339400.0,334040.0,328500.0,350500.0,357000.0,356000.0,365800.0,372500.0,361600.0,357400.0,350300.0,350400.0,343000.0,349800.0,345900.0,347000.0,348900.0,321000.0,215339.0,198700.0,247936.0,262515.0,227137.0,205526.0,212488.0,210906.0,177800.0,178200.0,165800.0,157600.0,153340.0,143900.0,147655.0,140870.0,132390.0,149100.0,146200.0,145700.0,147599.0,142800.0,142000.0,143149.0,142600.0,148084.0,145799.0,140110.0,132203.0,131310.0
533,3,'008,Albania,1717,'F1717,"Cereals, Total",5419,Yield,hg/ha,8452.0,9418.0,9823.0,10233.0,10373.0,13186.0,14641.0,15537.0,16398.0,16461.0,17536.0,17687.0,18430.0,19345.0,20289.0,24451.0,25501.0,26202.0,26348.0,23640.0,25006.0,27152.0,30460.0,29663.0,29006.0,29903.0,29656.0,28750.0,29617.0,27944.0,20719.0,21586.0,26850.0,24609.0,28415.0,24509.0,28333.0,28721.0,27981.0,31751.0,30308.0,32919.0,31857.0,34663.0,34621.0,36026.0,37284.0,40812.0,43153.0,47618.0,47507.0,48838.0,49498.0,48926.0,48738.0,47164.0,48130.0,48405.0,50382.0,52092.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76938,5817,'902,Net Food Importing Developing Countries,108,'01199.90,Cereals n.e.c.,5419,Yield,hg/ha,5699.0,5473.0,5631.0,5765.0,5929.0,5835.0,5929.0,6057.0,6003.0,6026.0,6029.0,6166.0,7170.0,6671.0,6769.0,6843.0,7093.0,7

Map of productiono of cereal

In [59]:
all_df[
    (all_df['Item Code'] == 1717) &\
    (all_df['Area Code'] <= 300) &\
    (all_df['Element'] == 'Yield')
][
    ['Area', 'Item', 'Element', 'Unit', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020']
].to_csv("./data/processed/cereal_yield_per_country_2015-2020.csv")